In [5]:
#!pip install tqdm
#!pip install pandas
#!pip install selenium

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
import pandas as pd
import os
import re
import time
import sys

DRIVER_PATH = './chromedriver.exe'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

base_url = "https://www.oxfordlearnersdictionaries.com/topic/"
BASE_PATH = "./oxford-vocab/"

if not os.path.isdir(BASE_PATH):
    os.mkdir(BASE_PATH)

In [6]:
r = requests.get(base_url)

if r.status_code == requests.codes.ok:
    soup = BeautifulSoup(r.text, 'html.parser')

topic_links = []
topics = soup.find_all("div", class_="topic-box")
for topic in topics:
    link = topic.find("a").get("href")
    topic_links.append(link)

In [7]:
def vocab_scraping(url, PATH):
    driver.get(url)
    time.sleep(2)
    
    word_list = []
    word_list_box = driver.find_element_by_xpath('.//ul[@class="top-g"]')
    word_list_box_elements = word_list_box.find_elements_by_xpath('.//li')
    
    for ele in word_list_box_elements:
        if ele.get_attribute("class") != "hidden": 
            vocab = ele.find_element_by_xpath('.//a').text
            pos = ele.find_element_by_xpath('.//span[@class="pos"]').text
            level = ele.find_element_by_xpath('.//span[@class="belong-to"]').text
            word_list.append((vocab,pos,level))
    print(f"Found {len(word_list)} vocabs.")
    
    word_list = sorted(word_list, key=lambda x: x[2])
    df = pd.DataFrame(word_list, columns=["vocab", "pos" ,"CEFR"])
    df.to_csv(PATH, index=False)

In [8]:
with tqdm(total=len(topic_links), file=sys.stdout) as pbar:
    for link_ in topic_links:
        pbar.set_description('processing: ' + link_[link_.find("category/")+len("category/"):])
        driver.get(link_)

        topic = driver.find_element_by_xpath('//h1[@class="topic_label"]').text

        TOPIC_PATH = BASE_PATH + topic
        if not os.path.isdir(TOPIC_PATH):
            os.mkdir(TOPIC_PATH)
        
        link_path_temp = []
        small_topic_boxes = driver.find_elements_by_xpath('//div[@class="topic-box topic-box-secondary"]')
        for topic_box in small_topic_boxes:
            topic_box_heading = topic_box.find_element_by_xpath('.//a[@class="topic-box-secondary-heading"]')

            topic_box_name = re.sub("\(see all\)", "", topic_box_heading.text)
            topic_box_name = re.sub(":", "", topic_box_name)

            SMALL_TOPIC_PATH = os.path.join(BASE_PATH + topic, topic_box_name)
            if not os.path.isdir(SMALL_TOPIC_PATH):
                os.mkdir(SMALL_TOPIC_PATH)

            link = topic_box_heading.get_attribute('href')
            link_path_temp.append((link, os.path.join(SMALL_TOPIC_PATH, "all.csv")))
            
            topic_box_elements = topic_box.find_elements_by_xpath('.//div[@class="l3"]/a')
            for element in topic_box_elements:
                link = element.get_attribute("href")
                link_path_temp.append((link, os.path.join(SMALL_TOPIC_PATH, element.text + ".csv")))
        
        
        for link, path in link_path_temp:
            vocab_scraping(link, path)
        
        pbar.update(1)

processing: animals_1:   0%|          | 0/18 [00:00<?, ?it/s]Found 530 vocabs.
Found 55 vocabs.
Found 10 vocabs.
Found 17 vocabs.
Found 68 vocabs.
Found 51 vocabs.
Found 19 vocabs.
Found 52 vocabs.
Found 37 vocabs.
Found 58 vocabs.
Found 154 vocabs.
Found 9 vocabs.
Found 264 vocabs.
Found 37 vocabs.
Found 6 vocabs.
Found 203 vocabs.
Found 18 vocabs.
Found 147 vocabs.
Found 98 vocabs.
Found 11 vocabs.
Found 11 vocabs.
Found 27 vocabs.
Found 119 vocabs.
Found 119 vocabs.
Found 11 vocabs.
Found 4 vocabs.
Found 63 vocabs.
Found 12 vocabs.
Found 8 vocabs.
Found 10 vocabs.
Found 11 vocabs.
processing: appearance_1:   6%|▌         | 1/18 [04:57<1:24:25, 297.98s/it]Found 538 vocabs.
Found 110 vocabs.
Found 47 vocabs.
Found 54 vocabs.
Found 135 vocabs.
Found 37 vocabs.
Found 13 vocabs.
Found 29 vocabs.
Found 39 vocabs.
Found 74 vocabs.
Found 299 vocabs.
Found 40 vocabs.
Found 29 vocabs.
Found 84 vocabs.
Found 29 vocabs.
Found 63 vocabs.
Found 54 vocabs.
Found 566 vocabs.
Found 32 vocabs.
Found 

FileNotFoundError: [Errno 2] No such file or directory: './oxford-vocab/Culture\\TV, radio and news\\Watching TV/Listening to the radio.csv'